In [10]:
import nltk
from textblob import TextBlob as tb
import math
import io
import codecs
from __future__ import division
import numpy
import pandas as pd
import matplotlib
%matplotlib inline 


In [83]:
import sys
import re
import os
import string
os.chdir('/users/asheets/Documents/Work_Computer_new/Work_Computer/Grad_School/PREDICT_453/Notebooks/DSI/')

all_docs = []
d = 24

cachedStopWords = nltk.corpus.stopwords.words('english')
pattern = re.compile(r'\b(' + r'|'.join(cachedStopWords) + r')\b\s*')

for i in range(16):
    doc_name = 'DSI' + str(d) + '.txt'
    try:
        with open(doc_name, 'r') as f:
            sample = f.read()
        sample = sample.decode('utf-8')
        sample = sample.lower()
        sample = ''.join([i for i in sample if not i.isdigit()])
        sample = pattern.sub('', sample)
        sample = "".join(l for l in sample if l not in string.punctuation)
        sample2 = " ".join(k for k in tb(sample).noun_phrases)
        all_docs.append({'DSInum': d, 'raw_text': sample, 'noun_phrases_only':
        sample2})
        d = d + 1
    except IOError:
        d = d + 1
        pass

RTV = pd.read_csv('RTV.txt')
    


                      administration
0                     americanEnergy
1                               bank
2                        barackObama
3                             border
4                             bridge
5                           business
6                           campaign
7                      climateChange
8                           congress
9                       conservative
10                         democracy
11                          democrat
12                   democraticParty
13                           economy
14                          election
15                            energy
16                       environment
17                        government
18                              hack
19                    hillaryClinton
20                   houseRepublican
21                         immigrant
22                       immigration
23                intelligenceAgency
24                  keystonePipeline
25                          lawmaker
2

In [81]:
#http://stevenloria.com/finding-important-words-in-a-document-using-tf-idf/

#computes "term frequency" which is the number of times a word appears in a document blob, 
#normalized by dividing by the total number of words in blob. 
def tf(word, blob):
    return blob.words.count(word) / len(blob.words)

#returns the number of documents containing word. A generator expression is passed to the sum() function.
def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob.words)

#computes "inverse document frequency" which measures how common a word is among all documents in bloblist. 
#The more common a word is, the lower its idf. 
#We take the ratio of the total number of documents to the number of documents containing word, 
#then take the log of that. Add 1 to the divisor to prevent division by zero.
def idf(word, bloblist):
     return math.log(len(bloblist) / float(n_containing(word, bloblist)))

#computes the TF-IDF score. It is simply the product of  tf and idf.
def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

In [18]:
#compare all three articles
blob1 = tb(sample1)
blob2 = blob1.noun_phrases
term_freq = [tf(word,blob1) for word in blob1.words]
tf_df1 = pd.DataFrame({'word': blob1.words, 'doc1_term_freq': term_freq})

blob2 = tb(sample2)
term_freq = [tf(word,blob2) for word in blob2.words]
tf_df2 = pd.DataFrame({'word': blob2.words, 'doc2_term_freq': term_freq})

blob3 = tb(sample3)
term_freq = [tf(word,blob3) for word in blob3.words]
tf_df3 = pd.DataFrame({'word': blob3.words, 'doc3_term_freq': term_freq})

all_tf = pd.merge(pd.merge(tf_df1,tf_df2,on='word',how='outer'),tf_df3,on='word',how='outer').drop_duplicates()

bloblist = [blob1,blob2,blob3]

docs_containing1 = pd.DataFrame({'word': blob1.words, 'doc_freq': [n_containing(word,bloblist) for word in blob1.words]})
docs_containing2 = pd.DataFrame({'word': blob2.words, 'doc_freq': [n_containing(word,bloblist) for word in blob2.words]})
docs_containing3 = pd.DataFrame({'word': blob3.words, 'doc_freq': [n_containing(word,bloblist) for word in blob3.words]})

doc_freq = pd.concat([docs_containing1,docs_containing2,docs_containing3]).drop_duplicates().reset_index(drop=True)
df = pd.merge(doc_freq,all_tf,on='word',how='inner')
df = df.sort_values("doc_freq",ascending=False)

df['intermediate_calc'] = (len(bloblist) / df['doc_freq']).astype(float)
df['idf'] = df['intermediate_calc'].apply(math.log)
df = df.sort_values("idf",ascending=True)


scores1 = pd.DataFrame({'word': blob1.words, 'tf_idf_doc1': [tfidf(word, blob1, bloblist) for word in blob1.words]})
scores2 = pd.DataFrame({'word': blob2.words, 'tf_idf_doc2': [tfidf(word, blob2, bloblist) for word in blob2.words]})
scores3 = pd.DataFrame({'word': blob3.words, 'tf_idf_doc3': [tfidf(word, blob3, bloblist) for word in blob3.words]})

all_tfidf= pd.merge(pd.merge(scores1,scores2,on='word',how='outer'),scores3,on='word',how='outer').drop_duplicates()
all_tfidf = all_tfidf[['word', 'tf_idf_doc1', 'tf_idf_doc2', 'tf_idf_doc3']]
all_tfidf = all_tfidf.sort_values(by=['tf_idf_doc1', 'tf_idf_doc2','tf_idf_doc3'], ascending=[False,False,False])
all_tfidf.head(n=6)



,word,tf_idf_doc1,tf_idf_doc2,tf_idf_doc3
1981,future,0.014967,NaN,NaN
283,energy,0.011048,0.008275,NaN
782,creation,0.005987,NaN,NaN
805,economic,0.005987,NaN,NaN
809,companies,0.005987,NaN,NaN
811,workers,0.005987,NaN,NaN


In [19]:
##Compare just the two articles we know to be similar
two_tf = pd.merge(tf_df1,tf_df2,on='word',how='outer').drop_duplicates()

bloblist = [blob1,blob2]

docs_containing1 = pd.DataFrame({'word': blob1.words, 'doc_freq': [n_containing(word,bloblist) for word in blob1.words]})
docs_containing2 = pd.DataFrame({'word': blob2.words, 'doc_freq': [n_containing(word,bloblist) for word in blob2.words]})

doc_freq = pd.concat([docs_containing1,docs_containing2]).drop_duplicates().reset_index(drop=True)
df = pd.merge(doc_freq,two_tf,on='word',how='inner')
df = df.sort_values("doc_freq",ascending=False)

df['intermediate_calc'] = (len(bloblist) / df['doc_freq']).astype(float)
df['idf'] = df['intermediate_calc'].apply(math.log)
df = df.sort_values("idf",ascending=False)

scores1 = pd.DataFrame({'word': blob1.words, 'tf_idf_doc1': [tfidf(word, blob1, bloblist) for word in blob1.words]})
scores2 = pd.DataFrame({'word': blob2.words, 'tf_idf_doc2': [tfidf(word, blob2, bloblist) for word in blob2.words]})

two_tfidf= pd.merge(scores1,scores2,on='word',how='outer').drop_duplicates()
two_tfidf = two_tfidf[['word', 'tf_idf_doc1', 'tf_idf_doc2']]
two_tfidf = two_tfidf.sort_values(by=['tf_idf_doc1', 'tf_idf_doc2'], ascending=[False,False])
two_tfidf.head(n=6)


,word,tf_idf_doc1,tf_idf_doc2
501,future,0.009443,NaN
360,back,0.007555,NaN
242,new,0.005666,NaN
592,white,0.005666,NaN
595,house,0.005666,NaN
249,job,0.003777,NaN


In [20]:
#compare all three using all pre-defined functions
bloblist = [blob1,blob2,blob3]
for i, blob in enumerate(bloblist):
    print("Top words in document {}".format(i + 1))
    scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:10]:
        print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))


Top words in document 1
	Word: future, TF-IDF: 0.01497
	Word: energy, TF-IDF: 0.01105
	Word: four, TF-IDF: 0.00599
	Word: previous, TF-IDF: 0.00599
	Word: workers, TF-IDF: 0.00599
	Word: environment, TF-IDF: 0.00599
	Word: effects, TF-IDF: 0.00599
	Word: companies, TF-IDF: 0.00599
	Word: gore, TF-IDF: 0.00599
	Word: creation, TF-IDF: 0.00599
Top words in document 2
	Word: paris, TF-IDF: 0.01121
	Word: energy, TF-IDF: 0.00827
	Word: industries, TF-IDF: 0.00747
	Word: rose, TF-IDF: 0.00747
	Word: global, TF-IDF: 0.00747
	Word: oil, TF-IDF: 0.00747
	Word: per, TF-IDF: 0.00747
	Word: accord, TF-IDF: 0.00747
	Word: undo, TF-IDF: 0.00747
	Word: activist, TF-IDF: 0.00747
Top words in document 3
	Word: fbi, TF-IDF: 0.01836
	Word: us, TF-IDF: 0.01836
	Word: russia, TF-IDF: 0.0153
	Word: intelligence, TF-IDF: 0.01428
	Word: steele, TF-IDF: 0.01122
	Word: russian, TF-IDF: 0.0102
	Word: people, TF-IDF: 0.00918
	Word: investigation, TF-IDF: 0.00816
	Word: dossier, TF-IDF: 0.00714
	Word: cia, TF-IDF

In [22]:
#compare similar docs using all pre-defined functions
bloblist = [blob1,blob2]
for i, blob in enumerate(bloblist):
    print("Top words in document {}".format(i + 1))
    scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:10]:
        print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))

Top words in document 1
	Word: future, TF-IDF: 0.00944
	Word: back, TF-IDF: 0.00755
	Word: new, TF-IDF: 0.00567
	Word: house, TF-IDF: 0.00567
	Word: white, TF-IDF: 0.00567
	Word: four, TF-IDF: 0.00378
	Word: previous, TF-IDF: 0.00378
	Word: workers, TF-IDF: 0.00378
	Word: environment, TF-IDF: 0.00378
	Word: stop, TF-IDF: 0.00378
Top words in document 2
	Word: paris, TF-IDF: 0.00707
	Word: industries, TF-IDF: 0.00472
	Word: rose, TF-IDF: 0.00472
	Word: global, TF-IDF: 0.00472
	Word: oil, TF-IDF: 0.00472
	Word: per, TF-IDF: 0.00472
	Word: accord, TF-IDF: 0.00472
	Word: undo, TF-IDF: 0.00472
	Word: activist, TF-IDF: 0.00472
	Word: groups, TF-IDF: 0.00472
